# Cralwer for IMDB top 250

The following notebook was created for retrieving top 250 from [IMDB](https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc)



In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


In [11]:
# make request to the url
# https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc&start=1&ref_=adv_nxt
# set parameters for the request
url = 'https://www.imdb.com/search/title/?groups=top_250&sort=user_rating,desc'

# get the response from the url with english language
response = requests.get(url, headers={'Accept-Language': 'en-US, en;q=0.5'})

# parse the response
soup = BeautifulSoup(response.text, 'html.parser')

In [63]:
# instantiate empty lists
title = []
runtime = []
genre = []
rating = []
metascore = []
votes = []
gross = []
year = []
director = []
description = []

# create dictionary with all the list
movies = {
    'title': title, 
    'runtime': runtime, 
    'genre': genre, 
    'rating': rating, 
    'metascore': metascore, 
    'votes': votes, 
    'gross': gross, 
    'year': year, 
    'director': director, 
    'description': description
    }


In [ ]:

# from soup get all elements with class lister-item-content
movie_containers = soup.find_all('div', class_ = 'lister-item-content')



In [64]:
for movie in movie_containers:
    # from first movie_container get the title
    title.append(movie.h3.a.text)
    # from first movie_container get the runtime
    runtime.append(movie.find('span', class_ = 'runtime').text.split(' ')[0])
    # from first movie_container get the genre
    genre.append(movie.find('span', class_ = 'genre').text.strip())
    # from first movie_container get the rating
    rating.append(movie.strong.text)
    # boolean check for metascore
    if movie.find('span', class_ = 'metascore') is not None:
        # from first movie_container get the metascore
        metascore.append(movie.find('span', class_ = 'metascore').text)
    else:
        # from first movie_container get the metascore
        metascore.append(' ')
    # from first movie_container get the number of votes
    votes.append(movie.find('span', attrs = {'name':'nv'})['data-value'])
    # from first movie_container get the year
    year.append(movie.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.strip('()'))
    # from first movie_container get the description
    description.append(movie.find_all('p', class_ = 'text-muted')[1].text.strip())
    # from first movie_container get the director
    director.append(movie.find_all('p', class_ = '')[0].a.text)
    # from first movie_container get the gross
    gross.append(movie.find('p', class_ ='sort-num_votes-visible').find_all('span', attrs = {'name':'nv'})[1].attrs['data-value'])

In [65]:
# create dataframe from dictionary to variable df
df = pd.DataFrame(movies)
df

,title,runtime,genre,rating,metascore,votes,gross,year,director,description
0,The Shawshank Redemption,142,Drama,9.3,81,2663689,"28,341,469",1994,Frank Darabont,Two imprisoned men bond over a number of years...
1,The Godfather,175,"Crime, Drama",9.2,100,1845919,"134,966,411",1972,Francis Ford Coppola,The aging patriarch of an organized crime dyna...
2,The Dark Knight,152,"Action, Crime, Drama",9.0,84,2636663,"534,858,444",2008,Christopher Nolan,When the menace known as the Joker wreaks havo...
3,The Lord of the Rings: The Return of the King,201,"Action, Adventure, Drama",9.0,94,1836488,"377,845,905",2003,Peter Jackson,Gandalf and Aragorn lead the World of Men agai...
4,Schindler's List,195,"Biography, Drama, History",9.0,94,1348869,"96,898,818",1993,Steven Spielberg,"In German-occupied Poland during World War II,..."
5,The Godfather Part II,202,"Crime, Drama",9.0,90,1264376,"57,300,000",1974,Francis Ford Coppola,The early life and career of Vito Corleone in ...
6,12 Angry Men,96,"Crime, Drama",9.0,96,786626,"4,360,000",1957,Sidney Lumet,The jury in a New York City murder trial is fr...
7,Jai Bhim,164,"Crime, Drama, Mystery",8.9,,201197,236,2021,T.J. Gnanavel,When a tribal man is arrested for a case of al...
8,Pulp Fiction,154,"Crime, Drama",8.9,94,2038858,"107,928,762",1994,Quentin Tarantino,"The lives of two mob hitmen, a boxer, a gangst..."
9,Inception,148,"Action, Adventure, Sci-Fi",8.8,74,2336435,"292,576,195",2010,Christopher Nolan,A thief who steals corporate secrets through t...
